In [6]:
import glob
import json
from pathlib import Path

import matplotlib.pyplot as plt
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import numpy as np
import pandas as pd
from torch.utils.data import Dataset

## Датасет

Прежде чем разбираться с моделями, нам надо в первую очередь разобраться с тем, как грузить датасет. Давайте напишем класс в торче для этого.

In [ ]:
# Задаем преобразование изображений
size = 1024
train_list = [A.LongestMaxSize(size, interpolation=cv2.INTER_CUBIC),
              A.PadIfNeeded(size, size),
              ToTensorV2(transpose_mask=True),
              ]
eval_list = [A.LongestMaxSize(size, interpolation=cv2.INTER_CUBIC),
              A.PadIfNeeded(size, size),
              ToTensorV2(transpose_mask=True)]

transforms = {'train': A.Compose(train_list), 'test': A.Compose(eval_list)}


In [14]:
from get_dataset import EyeDataset
# Инициализируем датасет
dataset = EyeDataset("dataset/train_dataset_mc")



In [16]:
nums = {'nums': [x for x in range(len(dataset))]}
dataset_df = pd.DataFrame(nums)
dataset_df.to_csv('dataset/nums.csv', sep=';', index_label='ind')

In [4]:
def get_df(dataset: EyeDataset, df: pd.DataFrame) -> pd.DataFrame:

    for ind in range(len(dataset)):
        try:
            new_row = {'image': dataset[ind]['image'], 'mask': dataset[ind]['mask']}
            df = df.append(new_row, ignore_index=True)
        except:
            continue

    yield df

In [7]:
dataset_df = pd.DataFrame(columns=['image', 'mask'])
#dataset_df = get_df(dataset, dataset_df)


In [6]:
#dataset_df

<generator object get_df at 0x7f35e0e05e70>

In [ ]:
test_dict = {'image': dataset['image'], 'mask': dataset['mask']}
dataset_df = pd.DataFrame(test_dict)
dataset_df

In [ ]:
img_arr = dataset_df.loc[0, 'image']
mask_arr = dataset_df.loc[0, 'mask']
print(dataset[0]['image_dir'])

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(25,25))
axes[0].imshow(img_arr)
axes[1].imshow(mask_arr[..., 0])
plt.show

In [ ]:
import PIL
from PIL import Image

im = PIL.Image.fromarray(np.uint8(img_arr))

In [ ]:
im1 = Image.open('dataset/train_dataset_mc/342.png')
im2 = Image.fromarray(np.uint8(mask_arr))


In [ ]:
im1.mode, im2.mode

In [ ]:
im2_1 = im2.convert('RGB')

In [ ]:
blended = Image.blend(im1, im2_1, alpha=0.99)
blended.save("blended.png")

In [ ]:
im2_1.show()

In [ ]:
cv2.imwrite('cv2.png', img_arr.astype(np.uint8))

In [ ]:
def keras_generator(gen_df: pd.DataFrame, batch_size: int):
    while True:
        x_batch = []
        y_batch = []
        
        for i in range(batch_size):
            img, mask = gen_df.sample(1).values[0]
            
            img = cv2.resize(img, (256,256))
            mask = cv2.resize(mask, (256,256))

            x_batch += [img]
            y_batch += [mask[..., 0]]

        x_batch = np.array(x_batch)
        y_batch = np.array(y_batch)

        yield x_batch, np.expand_dims(y_batch, -1)

In [ ]:
for x, y in keras_generator(dataset_df, 16):
    break
x.shape, y.shape